In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.stats import percentileofscore
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
def loadPrices():
    fn="../prices.txt"
    global nt, nInst
    df=pd.read_csv(fn, sep=r'\s+', header=None, index_col=None)
    (nt,nInst) = df.shape
    return nt, nInst, (df.values).T

In [ ]:
# import data
(nt, nInst, prcAll) = loadPrices()
prcTest = prcAll[:, :450]
prcCheck = prcAll[:, 450:600]
prcEval = prcAll[:, 600:]

In [ ]:
# Summary: What Pearson Correlation on Log Returns Really Means
# You're asking: "When Asset A moves up or down more than usual, does Asset B also tend to move in the same direction, at the same time, and by how much?"

# By using log returns instead of prices: You're removing trend and scale, and looking purely at the rhythm of movement between two instruments.
def comp_pearson(p_a, p_b):
    log_returns_A = np.diff(np.log(p_a))
    log_returns_B = np.diff(np.log(p_b))

    # -------------------------------------
    # Step 2: Compute Pearson Correlation
    # -------------------------------------
    pearson_np = np.corrcoef(log_returns_A, log_returns_B)[0, 1]

    return pearson_np   


def compute_correlation_matrix(prc_matrix, smooth=False, smooth_window=5):
    """
    Computes a correlation matrix of log returns, optionally smoothed.

    Parameters:
        prc_matrix (ndarray): (instruments, time) closing price matrix
        smooth (bool): Whether to smooth log returns before correlation
        smooth_window (int): Window size for moving average smoothing

    Returns:
        corr_matrix (ndarray): (instruments, instruments) Pearson correlation matrix
    """
    # Step 1: Compute log returns
    log_prices = np.log(prc_matrix)
    log_returns = np.diff(log_prices, axis=1)

    # Step 2: Optional smoothing (moving average over time axis)
    if smooth:
        kernel = np.ones(smooth_window) / smooth_window
        smoothed_returns = np.array([
            np.convolve(r, kernel, mode='valid')
            for r in log_returns
        ])
    else:
        smoothed_returns = log_returns

    # Step 3: Compute Pearson correlation matrix
    corr_matrix = np.corrcoef(smoothed_returns)
    return corr_matrix

def plot_correlation_heatmap(corr_matrix, title="Correlation Matrix Heatmap"):
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        corr_matrix,
        cmap='coolwarm',       # red = negative, blue = positive
        vmin=-1, vmax=1,       # range of correlation values
        center=0,
        square=True,
        cbar_kws={'shrink': .6},
        xticklabels=False,
        yticklabels=False
    )
    plt.title(title, fontsize=16)
    plt.tight_layout()
    plt.show()

def get_top_correlated_pairs(corr_matrix, top_n=5):
    """
    Returns the top N most strongly positively correlated instrument index pairs
    from a correlation matrix (excluding diagonal/self-correlations).

    Parameters:
        corr_matrix (ndarray): A symmetric NxN correlation matrix
        top_n (int): Number of top pairs to return

    Returns:
        List of tuples: (instrument_i, instrument_j, correlation)
    """
    # Get indices of upper triangle, excluding diagonal
    n = corr_matrix.shape[0]
    upper_tri_indices = np.triu_indices(n, k=1)

    # Extract the upper triangle values and corresponding index pairs
    corr_values = corr_matrix[upper_tri_indices]
    index_pairs = list(zip(upper_tri_indices[0], upper_tri_indices[1]))

    # Sort by correlation values descending
    sorted_pairs = sorted(zip(index_pairs, corr_values), key=lambda x: x[1], reverse=True)

    # Return top N
    return [(i, j, round(val, 4)) for ((i, j), val) in sorted_pairs[:top_n]]

In [ ]:
# Checking individual pearson corr  

inst_a = 10
inst_b = 11
print(comp_pearson(prcTest[inst_a, :], prcTest[inst_b, :]))

In [ ]:
corr_matrix = compute_correlation_matrix(prcTest, True, 25)
plot_correlation_heatmap(corr_matrix)


In [ ]:
top5 = get_top_correlated_pairs(corr_matrix, top_n=5)
for i, j, val in top5:
    print(f"Instrument {i} & Instrument {j} → Correlation: {val}")

In [ ]:
def plot_top_correlated_pairs(prc_matrix, corr_matrix, top_n=5, use_returns=False, smooth=False, smooth_window=5):
    """
    Plots the top N most correlated instrument pairs from a correlation matrix.

    Parameters:
        prc_matrix (ndarray): (instruments, time) closing price matrix
        corr_matrix (ndarray): (instruments, instruments) correlation matrix
        top_n (int): Number of top correlated pairs to plot
        use_returns (bool): If True, plot log returns instead of prices
        smooth (bool): Whether to apply moving average smoothing
        smooth_window (int): Window size for smoothing (in days)
    """
    # Step 1: Get top correlated index pairs
    n = corr_matrix.shape[0]
    upper = np.triu_indices(n, k=1)
    corr_values = corr_matrix[upper]
    index_pairs = list(zip(upper[0], upper[1]))
    sorted_pairs = sorted(zip(index_pairs, corr_values), key=lambda x: x[1], reverse=True)
    top_pairs = sorted_pairs[:top_n]

    # Step 2: Compute returns or use prices
    if use_returns:
        log_prices = np.log(prc_matrix)
        data = np.diff(log_prices, axis=1)
        ylabel = "Log Returns"
    else:
        data = prc_matrix
        ylabel = "Price"

    # Step 3: Optional smoothing
    if smooth:
        # Apply moving average across time axis (axis=1)
        kernel = np.ones(smooth_window) / smooth_window
        data = np.array([
            np.convolve(row, kernel, mode='valid')
            for row in data
        ])
        xlabel = f"Time (Days) [Smoothed, Window={smooth_window}]"
    else:
        xlabel = "Time (Days)"

    # Step 4: Plot each pair
    for idx, ((i, j), corr_val) in enumerate(top_pairs, 1):
        plt.figure(figsize=(10, 4))
        plt.plot(data[i], label=f"Instrument {i}", linewidth=2)
        plt.plot(data[j], label=f"Instrument {j}", linewidth=2, linestyle='--')
        plt.title(f"Top {idx}: Instruments {i} & {j} (Correlation = {corr_val:.4f})")
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.legend()
        plt.tight_layout()
        plt.show()

plot_top_correlated_pairs(prcTest, corr_matrix, 5, False)

In [ ]:

def predict_instrument_returns(prc_matrix, corr_matrix, target_idx):
    """
    Predict the log returns of the target instrument as a weighted sum of
    other instruments' log returns, weighted by their correlation with the target.

    Parameters:
        prc_matrix (ndarray): shape (num_instruments, num_days), closing prices
        corr_matrix (ndarray): shape (num_instruments, num_instruments), correlation matrix of returns
        target_idx (int): index of the instrument to predict

    Returns:
        predicted_returns (ndarray): predicted log returns for the target instrument, shape (num_days-1,)
    """
    n = prc_matrix.shape[0]
    assert corr_matrix.shape == (n, n), "Correlation matrix shape mismatch"

    # Calculate log returns
    log_prices = np.log(prc_matrix)
    log_returns = np.diff(log_prices, axis=1)  # shape (n, num_days-1)

    # Get weights from correlation matrix, zero self-correlation
    weights = corr_matrix[target_idx].copy()
    weights[target_idx] = 0

    # Normalize weights by sum of absolute values to keep scale
    sum_weights = np.sum(np.abs(weights))
    if sum_weights != 0:
        weights /= sum_weights

    # Weighted sum of other instruments' returns to predict target returns
    predicted_returns = weights @ log_returns  # shape (num_days-1,)

    return predicted_returns


def reconstruct_price_from_returns(start_price, returns):
    """
    Reconstruct price series from starting price and log returns.

    Parameters:
        start_price (float): initial price before returns start
        returns (ndarray): log returns array shape (num_days-1,)

    Returns:
        prices (ndarray): reconstructed price series shape (num_days,)
    """
    cum_log_return = np.cumsum(returns)
    prices = start_price * np.exp(cum_log_return)
    prices = np.insert(prices, 0, start_price)  # include starting price
    return prices


def evaluate_predicted_returns(prc_matrix, predicted_returns, target_idx):
    """
    Compare predicted returns with actual returns and compute metrics.

    Parameters:
        prc_matrix (ndarray): (num_instruments, num_days) price matrix
        predicted_returns (ndarray): predicted log returns for target instrument, shape (num_days-1,)
        target_idx (int): index of the instrument being predicted

    Returns:
        metrics (dict): mse, mae, r2 scores
    """
    # Calculate actual log returns
    log_prices = np.log(prc_matrix)
    actual_returns = np.diff(log_prices[target_idx])

    # Metrics
    mse = mean_squared_error(actual_returns, predicted_returns)
    mae = mean_absolute_error(actual_returns, predicted_returns)
    r2 = r2_score(actual_returns, predicted_returns)

    print(f"Evaluation for Instrument {target_idx}:")
    print(f"Mean Squared Error (MSE): {mse:.6f}")
    print(f"Mean Absolute Error (MAE): {mae:.6f}")
    print(f"R² Score: {r2:.4f}")

    # Plot actual vs predicted returns
    plt.figure(figsize=(12, 5))
    plt.plot(actual_returns, label='Actual Returns', alpha=0.7)
    plt.plot(predicted_returns, label='Predicted Returns', alpha=0.7)
    plt.title(f'Actual vs Predicted Returns for Instrument {target_idx}')
    plt.xlabel('Days')
    plt.ylabel('Log Return')
    plt.legend()
    plt.show()

    return {'mse': mse, 'mae': mae, 'r2': r2}
r2 = list()

data = prcCheck
for target_idx in range(50):
    predicted_ret = predict_instrument_returns(data, corr_matrix, target_idx)

    # Reconstruct predicted prices starting from actual first price
    predicted_prices = reconstruct_price_from_returns(data[target_idx, 0], predicted_ret)

    plt.plot(data[target_idx], label="Actual Prices")
    plt.plot(predicted_prices, label="Predicted Prices from Correlation")
    plt.legend()
    plt.title(f"Actual vs Predicted Prices for Instrument {target_idx}")
    plt.show()
    predicted_ret = predict_instrument_returns(data, corr_matrix, target_idx)
    metrics = evaluate_predicted_returns(data, predicted_ret, target_idx)
    r2.append(metrics['r2'])
print(r2)
    

In [ ]:
def top_n_with_indices(arr, n=5):
    arr = np.array(arr)
    # Get indices of top n values
    top_indices = np.argsort(arr)[-n:][::-1]
    top_values = arr[top_indices]
    return list(zip(top_indices, top_values))
top5 = top_n_with_indices(r2, 5)